In [1]:
library(readr)
library(glmnet)
library(tidyr)
library(dplyr, warn=F)
library(glmnet)

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-2


Attaching package: ‘tidyr’

The following object is masked from ‘package:Matrix’:

    expand



In [2]:
part_df = readr::read_tsv('data/partition.tsv.gz')
dwpc_df = readr::read_tsv('data/dwpc.tsv.gz')

In [3]:
spread_df = part_df %>%
  dplyr::inner_join(
    dwpc_df %>%
    dplyr::select(-path_count) %>%
    tidyr::spread(key = metapath, value = dwpc)
  )

Joining by: c("compound_id", "disease_id")


In [4]:
x = spread_df %>%
  dplyr::select(-(compound_id:percentile)) %>%
  as.matrix()
y = spread_df$indication

In [5]:
lasso = hetior::glmnet_train(x, y, alpha=1)

In [6]:
# Positive coefficients
lasso$coef_df %>%
  dplyr::arrange(desc(zcoef)) %>%
  dplyr::filter(zcoef > 0)

,feature,coef,zcoef
1,CcSEcCiD,32.69709,2.391688
2,CiDiCiD,37.85164,2.290817
3,CtGtCiD,15.25964,1.136177
4,CsCsCiD,8.675185,0.7225817
5,CtGaD,21.45194,0.6782444
6,CiDaGaD,9.698123,0.4539967
7,CiDsD,6.792177,0.4104101
8,CsCiD,3.863816,0.346301
9,CiDlAlD,6.443075,0.2873377
10,CiDsDsD,2.607129,0.1275331


In [7]:
lasso$vtm[c('auroc', 'auprc')]

$auroc
[1] 0.9665569

$auprc
[1] 0.9236064

In [9]:
pred_df = spread_df %>%
  dplyr::select(compound_id:percentile)
pred_df$y_pred = lasso$y_pred
pred_df = pred_df %>%
  dplyr::arrange(disease_name, desc(y_pred))
pred_df %>% readr::write_csv('data/predictions.csv')